# Group Surveys Feature Determination

## Preparation

### Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

## Function Definition

### Min-Max Scaler

In [3]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scaling_df(df):
    scaler = MinMaxScaler()
    scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return scaled_df


## Use of Function

In [4]:
in_file= r'/content/drive/MyDrive/Projects/tps/finals/data/3_individual_features.xlsx'
out_file= r'/content/drive/MyDrive/Projects/tps/finals/data/6_model_regression_full_features.xlsx'

In [5]:
df= pd.read_excel(in_file, index_col='Id')

In [6]:
df.head(12)

,indiv_spoken_time,indiv_spoken_time_ratio,average_turn_duration,average_turn_duration_ratio,avg_time_without_speaking,avg_time_without_speaking_ratio,max_time_without_speaking,max_time_without_speaking_ratio,num_turns,num_turns_ratio,avg_turns_without_speaking,avg_turns_without_speaking_ratio,max_turns_without_speaking,max_turns_without_speaking_ratio,num_words,num_words_ratio,avg_words_turn,avg_words_turn_ratio,max_words_turn,max_words_turn_ratio,speech_neu,speech_ang,speech_hap,speech_sad,text_joy,text_anger,text_fear,text_sadness,messages_sent,messages_total,contribution_index,ego_art,ego_nudges,alter_art,alter_nudges,sentiment_avg,emotionality_avg,complexity_avg,influence_message_avg,influence_total_in,influence_message_avg_in,influence_total,contribution_index_oscillation,activity_entanglement,ALTERNATIVE_REALITIES_Treehugger,ALTERNATIVE_REALITIES_Fatherlander,ALTERNATIVE_REALITIES_Spiritualism,ALTERNATIVE_REALITIES_Nerd,EMOTIONS_Fear,EMOTIONS_Happy,EMOTIONS_Sad,EMOTIONS_Anger,Groupflow_Beeflow,Groupflow_Leechflow,Groupflow_Antflow,ethical_likelihood,financial_likelihood,health_likelihood,recreational_likelihood,social_likelihood,total_likelihood,ethical_perceived,financial_perceived,health_perceived,recreational_perceived,social_perceived,total_perceived,O,C,E,A,N,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,dummy_question1,dummy_question2,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,conservation,transcendence,theory,coeval,project
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,526,0.148336,7.850746,0.126865,56.210746,0.165958,451.54,0.264343,67,0.331683,1.985075,0.093801,7,0.090909,978,0.170800,14.597015,0.127952,68,0.078613,0.832695,0.137502,0.026347,0.003456,0.298809,0.236814,0.232361,0.232015,34,187,-0.64,1.332.855.889,2.468.578.279,1.234.255.833,1.903.921.545,0.303561,0.215562,9.179.501.577,0.346125353,1.227.594.788,0.346135338,1.730.626.767,1,0.487649,0.472084,0.083889,0.089140,0.354886,0.236101,0.407447,0.251288,0.105165,0.497477,0.156358,0.346165,4.500000,3.500000,4.166667,3.166667,3.500000,3.766667,4.833333,5.666667,5.666667,5.833333,4.833333,5,0.600000,0.716667,0.600000,0.633333,0.583333,27,23,19,18,20,1,5,7,6,5,7,5,4,5,7,8,7,2.57,-1.78,3.44,0.90,8.0
2,2648,0.746757,33.948718,0.548598,21.791169,0.064337,109.24,0.063952,78,0.386139,1.589744,0.075120,9,0.116883,3816,0.666434,48.923077,0.428842,623,0.720231,0.795958,0.155654,0.045315,0.003074,0.292950,0.239695,0.227768,0.239587,75,228,-0.34,7.778.626.875,1.363.151.848,9.695.611.111,3.791.190.505,0.250396,0.236507,9.308.010.889,0.428515776,0.997035569,0.231981331,1.988.107.685,2,0.398994,0.438515,0.040232,0.107725,0.413528,0.215535,0.417419,0.208064,0.158982,0.300955,0.220056,0.478989,2.000000,1.500000,1.000000,5.166667,5.500000,3.033333,4.000000,6.666667,6.500000,5.500000,3.000000,5,0.533333,0.666667,0.700000,0.616667,0.633333,22,28,11,7,11,0,5,2,6,4,4,7,8,8,0,5,4,-0.24,-0.33,3.64,0.91,8.0
3,105,0.029611,8.076923,0.130520,57.649231,0.170205,141.72,0.082966,13,0.064356,6.461538,0.305327,15,0.194805,267,0.046629,20.538462,0.180033,70,0.080925,0.789103,0.161515,0.037497,0.011885,0.297727,0.238513,0.230916,0.232844,38,191,-0.60,1.006.371.139,2.010.089.278,9.903.272.639,2.270.121.068,0.381860,0.254014,9.034.121.152,0.236068168,0.866593847,0.199153824,103.894.739,2,0.501416,0.157377,0.028980,0.053947,0.759697,0.245048,0.410218,0.190879,0.153855,0.226931,0.238211,0.534858,4.500000,3.500000,2.666667,6.000000,6.000000,4.533333,5.000000,5.500000,5.500000,4.333333,2.833333,5,0.566667,0.683333,0.716667,0.533333,0.716667,23,28,9,6,7,1,5,6,7,6,8,8,8,6,3,2,6,-0.52,-2.03,3.98,0.90,8.0
4,162,0.045685,6.480000,0.104714,159.421600,0.470679,836.46,0.489685,25,0.123762,6.600000,0.311870,29,0.376623,394,0.068809,15.760000,0.138147,61,0.070520,0.712697,0.196289,0.077861,0.013153,0.300100,0.235307,0.232877,0.231716,27,180,-0.70,1.197.692.111,2.333.088.249,7.537.312.014,1.568.276.525

In [7]:
df.shape

(57, 94)

In [8]:
# Strip off leading and trailing whitespace from non-numeric columns
df = df.select_dtypes(include=['int', 'float'])
df.reset_index(drop=True, inplace=True)

In [9]:
def create_random_df(df):
  limit0= 0
  limit1= 28 #28 Column: messages_sent
  limit2= 45 #45 Column: Groupflow_Antflow
  limit3= 81 #Index: 81 Column: transcendence

  # Generate two random integers within the range 1-8
  range_1 = random.sample(range(limit0, limit1), 2)

  # Generate two random integers within the range 9-16
  range_2 = random.sample(range(limit1, limit2), 2)

  # Generate two random integers within the range 17-25
  range_3 = random.sample(range(limit2, limit3), 2)

  # Combine all six integers into a single list
  column_indexes = range_1 + range_2 + range_3

  new_df = df.iloc[:, column_indexes].copy()
  data = pd.concat([new_df, df['theory']], axis=1)
  data = min_max_scaling_df(data)
  selected_columns = df.columns[column_indexes].to_list()

  return data,selected_columns

In [10]:
def create_random_df_by_columns(df,column_names):

  new_df = df[column_names].copy()
  data = pd.concat([new_df, df['theory']], axis=1)
  data = min_max_scaling_df(data)

  return data

In [11]:
#df= drop_bad_measured(df)


## Regression Models

In [12]:
def grid_search_cv(df):
  from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
  from sklearn.metrics import make_scorer, mean_squared_error, r2_score
  from sklearn.linear_model import LinearRegression
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import RandomForestRegressor

  # Split the data into training and test sets
  X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=42)

  # Define the list of models to evaluate
  models = [
      ('Linear Regression', LinearRegression()),
      ('Decision Tree', DecisionTreeRegressor()),
      ('Random Forest', RandomForestRegressor())
  ]

  # Define the grid of hyperparameters for each model
  param_grid = {
      'Linear Regression': {},
      'Decision Tree': {'max_depth': [None, 5, 10]},
      'Random Forest': {'n_estimators': [100, 200, 300]}
  }

  # Define the scoring metrics for evaluation
  scoring = {
      'RMSE': make_scorer(mean_squared_error, squared=False),
      'R2': make_scorer(r2_score)
  }

  # Perform grid search and cross-validation for each model
  results = {}
  for name, model in models:
      grid_search = GridSearchCV(model, param_grid[name], scoring=scoring, refit='RMSE', cv=5)
      grid_search.fit(X_train, y_train)
      
      # Cross-validation scores for all parameter combinations
      cv_results = cross_validate(grid_search.best_estimator_, X_train, y_train, scoring=scoring, cv=5)
      
      results[name] = {
          'best_params': grid_search.best_params_,
          'best_estimator': grid_search.best_estimator_,
          'cv_RMSE': cv_results['test_RMSE'].mean(),
          'cv_R2': cv_results['test_R2'].mean()
      }

  # Print the results for all models and parameter combinations
  for name, result in results.items():
      print("Model:", name)
      print("Best Parameters:", result['best_params'])
      print("CV RMSE:", result['cv_RMSE'])
      print("CV R2:", result['cv_R2'])
      print()

  # Train the best model on the entire training dataset
  best_model = min(results, key=lambda x: results[x]['cv_RMSE'])
  final_model = results[best_model]['best_estimator']
  final_model.fit(X_train, y_train)

  # Evaluate the performance of the best model on the test set
  y_pred = final_model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, y_pred)
  print("Best Model:", best_model)
  print("Test RMSE:", rmse)
  print("Test R2:", r2)


In [15]:
def try_regressions(df):

  # Step 2: Split the data into features (X) and the objective variable (y)
  X = df.iloc[:, :-1]  # All columns except the last one
  y = df.iloc[:, -1]   # Last column

  # Step 3: Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Step 4: Build and evaluate different OLS models
  models = []

  # Model 1: OLS using statsmodels
  model1 = sm.OLS(y_train, sm.add_constant(X_train))
  results1 = model1.fit()
  models.append(results1)

  # Model 2: OLS using scikit-learn (linear regression)
  from sklearn.linear_model import LinearRegression
  model2 = LinearRegression()
  model2.fit(X_train, y_train)
  models.append(model2)

  # Model 3: OLS using scikit-learn (Ridge regression)
  from sklearn.linear_model import Ridge
  model3 = Ridge(alpha=1.0)  # You can adjust the regularization parameter (alpha) as needed
  model3.fit(X_train, y_train)
  models.append(model3)

  # Step 5: Evaluate the models on the testing set
  for model in models:
      if isinstance(model, sm.regression.linear_model.RegressionResultsWrapper):
          X_test_with_constant = sm.add_constant(X_test)
          y_pred = model.predict(X_test_with_constant)
          r2 = model.rsquared
      else:
          y_pred = model.predict(X_test)
          r2 = model.score(X_test, y_test)

      # Calculate evaluation metrics (e.g., mean squared error)
      mse = np.mean((y_pred - y_test) ** 2)

      # Print the evaluation results
      print(model)
      print("Mean Squared Error:", mse)
      print("R-squared:", r2)
      print()



In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge

def try_regressions(df):
    # Step 2: Split the data into features (X) and the objective variable (y)
    X = df.iloc[:, :-1]  # All columns except the last one
    y = df.iloc[:, -1]   # Last column

    # Step 3: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 4: Build and evaluate different OLS models
    models = []

    # Model 1: OLS using scikit-learn (linear regression)
    model1 = LinearRegression()
    model1.fit(X_train, y_train)
    models.append(('Linear Regression', model1))

    # Model 2: OLS using scikit-learn (Ridge regression)
    model2 = Ridge(alpha=1.0)  # You can adjust the regularization parameter (alpha) as needed
    model2.fit(X_train, y_train)
    models.append(('Ridge Regression', model2))

    # Step 5: Evaluate the models using cross-validation
    for name, model in models:
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        mse = -scores.mean()
        r2 = model.score(X_test, y_test)

        # Print the evaluation results
        print(name)
        print("Mean Squared Error:", mse)
        print("R-squared:", r2)
        print()


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge

def try_regressions(df):
    # Step 2: Split the data into features (X) and the objective variable (y)
    X = df.iloc[:, :-1]  # All columns except the last one
    y = df.iloc[:, -1]   # Last column

    # Step 3: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 4: Build and evaluate different OLS models
    models = []

    # Model 1: OLS using scikit-learn (linear regression)
    model1 = LinearRegression()
    model1.fit(X_train, y_train)
    models.append(('Linear Regression', model1))

    # Model 2: OLS using scikit-learn (Ridge regression)
    model2 = Ridge(alpha=1.0)  # You can adjust the regularization parameter (alpha) as needed
    model2.fit(X_train, y_train)
    models.append(('Ridge Regression', model2))

    # Step 5: Evaluate the models using cross-validation
    results = {}

    for name, model in models:
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        mse = -scores.mean()
        r2 = model.score(X_test, y_test)

        # Store the evaluation results in a dictionary
        results[name] = {'Mean Squared Error': mse, 'R-squared': r2}

    return results


## Use of Functions

In [18]:
import pandas as pd
import numpy as np

results = []
indexes = []

for x in range(0, 100000):
    df_test, index = create_random_df(df)
    results.append(try_regressions(df_test))
    indexes.append(index)

data = []

for i, result in enumerate(results):
    for model_name, metrics in result.items():
        r2 = metrics['R-squared']
        mse = metrics['Mean Squared Error']
        data.append([model_name, r2, mse, indexes[i]])

df_results = pd.DataFrame(data, columns=['Model', 'R-squared', 'Mean Squared Error', 'Index'])

df_results = df_results.sort_values(by='R-squared', ascending=False)

df_results.head(10)

,Model,R-squared,Mean Squared Error,Index
64754,Linear Regression,0.894838,0.030272,"[max_turns_without_speaking, indiv_spoken_time..."
33572,Linear Regression,0.851039,0.055190,"[speech_ang, max_time_without_speaking, contri..."
99188,Linear Regression,0.839684,0.032252,"[text_sadness, max_turns_without_speaking, con..."
35126,Linear Regression,0.814960,0.036523,"[max_turns_without_speaking, text_joy, contrib..."
139420,Linear Regression,0.814419,0.024565,"[avg_words_turn_ratio, avg_turns_without_speak..."
36612,Linear Regression,0.813485,0.032413,"[text_sadness, num_words_ratio, Groupflow_Beef..."
48252,Linear Regression,0.797601,0.028449,"[text_joy, max_time_without_speaking_ratio, me..."
21586,Linear Regression,0.796825,0.029966,"[avg_words_turn, text_joy, ALTERNATIVE_REALITI..."
71926,Linear Regression,0.791876,0.026885,"[max_words_turn, average_turn_duration, contri..."
73528,Linear Regression,0.790335,0.038551,"[max_time_without_speaking, average_turn_durat..."


In [19]:
df_results.head(100).to_excel(out_file, index=False)

In [20]:
cols= ['text_anger', 'num_turns', 'contribution_index_oscillation', 'messages_sent', 'health_perceived', 'total_likelihood']
df_testing= create_random_df_by_columns(df,cols)
print(try_regressions(df_testing))

{'Linear Regression': {'Mean Squared Error': 0.021278770963762923, 'R-squared': 0.7863688588410277}, 'Ridge Regression': {'Mean Squared Error': 0.026921441960571176, 'R-squared': 0.46640281920896276}}
